# 03. 倍音解析

音の倍音構造を分析し、楽器の特徴や音色を理解します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks, get_window
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本的な倍音構造

In [ ]:
def create_harmonic_series(fundamental_freq, num_harmonics, amplitudes=None, sample_rate=44100, duration=1.0):
    """
    倍音列を生成
    """
    if amplitudes is None:
        # デフォルト: 1/n の減衰
        amplitudes = [1.0 / n for n in range(1, num_harmonics + 1)]
    
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    signal = np.zeros_like(t)
    
    for n, amp in enumerate(amplitudes, 1):
        harmonic_freq = fundamental_freq * n
        signal += amp * np.sin(2 * np.pi * harmonic_freq * t)
    
    return t, signal

# 異なる楽器の倍音構造をシミュレート
instruments = {
    'サイン波': [1.0],
    'のこぎり波': [1.0, 0.5, 0.33, 0.25, 0.2, 0.17, 0.14, 0.13],  # 1/n
    '方形波': [1.0, 0, 0.33, 0, 0.2, 0, 0.14, 0],  # 奇数倍音のみ
    'フルート風': [1.0, 0.1, 0.05, 0.02, 0.01],  # 基音が強い
    'バイオリン風': [1.0, 0.8, 0.6, 0.4, 0.3, 0.2, 0.15, 0.1],  # 豊富な倍音
    'トランペット風': [1.0, 0.7, 0.5, 0.6, 0.4, 0.3, 0.2, 0.15]  # 中高域が強い
}

fundamental = 220  # A3
sample_rate = 8000
duration = 1.0

plt.figure(figsize=(18, 15))

for i, (instrument, amplitudes) in enumerate(instruments.items()):
    # 倍音列を生成
    t, signal = create_harmonic_series(fundamental, len(amplitudes), amplitudes, sample_rate, duration)
    
    # FFT
    fft_result = fft(signal)
    frequencies = fftfreq(len(signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    amplitude_spectrum = np.abs(fft_result)
    
    # 時間波形
    plt.subplot(6, 3, i*3 + 1)
    plt.plot(t[:400], signal[:400], linewidth=2)
    plt.title(f'{instrument}（時間波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # 倍音構造（線形スケール）
    plt.subplot(6, 3, i*3 + 2)
    plt.plot(frequencies[positive_mask], amplitude_spectrum[positive_mask], 'b-', linewidth=2)
    
    # 倍音ピークをマーク
    for n, amp in enumerate(amplitudes, 1):
        harmonic_freq = fundamental * n
        if harmonic_freq < sample_rate / 2:  # ナイキスト周波数以下
            plt.axvline(x=harmonic_freq, color='r', linestyle='--', alpha=0.7)
            plt.text(harmonic_freq, max(amplitude_spectrum[positive_mask]) * 0.8, 
                    f'{n}', ha='center', fontsize=8, color='red')
    
    plt.title(f'{instrument}（周波数スペクトル）')
    plt.ylabel('振幅')
    plt.grid(True)
    plt.xlim(0, 2000)
    
    # 倍音構造（dBスケール）
    plt.subplot(6, 3, i*3 + 3)
    amplitude_db = 20 * np.log10(amplitude_spectrum[positive_mask] + 1e-10)
    plt.plot(frequencies[positive_mask], amplitude_db, 'g-', linewidth=2)
    
    # 倍音ピークをマーク
    for n, amp in enumerate(amplitudes, 1):
        harmonic_freq = fundamental * n
        if harmonic_freq < sample_rate / 2:
            plt.axvline(x=harmonic_freq, color='r', linestyle='--', alpha=0.7)
    
    plt.title(f'{instrument}（dBスケール）')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(0, 2000)
    
    if i == len(instruments) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(6, 3, i*3 + 2)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(6, 3, i*3 + 3)
        plt.xlabel('周波数 (Hz)')

plt.tight_layout()
plt.show()

## 2. 倍音検出アルゴリズム

In [ ]:
def detect_harmonics(signal, sample_rate, min_frequency=50, max_frequency=2000):
    """
    信号から倍音を検出
    """
    # FFT
    fft_result = fft(signal)
    frequencies = fftfreq(len(signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    
    freq_positive = frequencies[positive_mask]
    amplitude_positive = np.abs(fft_result[positive_mask])
    
    # 指定周波数範囲内のマスク
    range_mask = (freq_positive >= min_frequency) & (freq_positive <= max_frequency)
    freq_range = freq_positive[range_mask]
    amp_range = amplitude_positive[range_mask]
    
    # ピーク検出
    peaks, properties = find_peaks(amp_range, 
                                  height=np.max(amp_range) * 0.05,  # 最大値の5%以上
                                  distance=20)  # 最小距離
    
    peak_frequencies = freq_range[peaks]
    peak_amplitudes = amp_range[peaks]
    
    # 最も強いピークを基音と仮定
    fundamental_idx = np.argmax(peak_amplitudes)
    fundamental_freq = peak_frequencies[fundamental_idx]
    
    # 倍音関係を検出
    harmonics = []
    tolerance = 0.1  # 10%の誤差許容
    
    for freq, amp in zip(peak_frequencies, peak_amplitudes):
        # 何倍音かを計算
        harmonic_ratio = freq / fundamental_freq
        nearest_integer = round(harmonic_ratio)
        
        # 整数倍に近いかチェック
        if abs(harmonic_ratio - nearest_integer) < tolerance:
            harmonics.append({
                'harmonic_number': nearest_integer,
                'frequency': freq,
                'amplitude': amp,
                'amplitude_db': 20 * np.log10(amp + 1e-10)
            })
    
    # 倍音番号でソート
    harmonics.sort(key=lambda x: x['harmonic_number'])
    
    return {
        'fundamental_frequency': fundamental_freq,
        'harmonics': harmonics,
        'all_peaks': list(zip(peak_frequencies, peak_amplitudes)),
        'frequencies': freq_positive,
        'amplitudes': amplitude_positive
    }

# テスト信号で倍音検出
test_cases = [
    ('完全な倍音列', [1.0, 0.5, 0.33, 0.25, 0.2]),
    ('奇数倍音のみ', [1.0, 0, 0.33, 0, 0.2, 0, 0.14]),
    ('ノイズ付き倍音', [1.0, 0.7, 0.5, 0.3, 0.2]),
    ('不完全な倍音', [1.0, 0.6, 0.8, 0.2, 0.4])  # 3倍音が強い
]

fundamental = 150  # Hz

plt.figure(figsize=(16, 12))

for i, (case_name, amplitudes) in enumerate(test_cases):
    # 信号生成
    t, signal = create_harmonic_series(fundamental, len(amplitudes), amplitudes, sample_rate, duration)
    
    # ノイズを追加（ノイズ付きの場合）
    if 'ノイズ' in case_name:
        np.random.seed(42)
        signal += 0.1 * np.random.normal(0, 1, len(signal))
    
    # 倍音検出
    result = detect_harmonics(signal, sample_rate)
    
    # 結果表示
    plt.subplot(2, 2, i + 1)
    
    # スペクトル表示
    plt.plot(result['frequencies'], result['amplitudes'], 'b-', alpha=0.7, linewidth=1)
    
    # 検出された全ピーク
    all_peak_freqs, all_peak_amps = zip(*result['all_peaks'])
    plt.plot(all_peak_freqs, all_peak_amps, 'go', markersize=6, alpha=0.7, label='検出ピーク')
    
    # 倍音として認識されたピーク
    harmonic_freqs = [h['frequency'] for h in result['harmonics']]
    harmonic_amps = [h['amplitude'] for h in result['harmonics']]
    harmonic_numbers = [h['harmonic_number'] for h in result['harmonics']]
    
    plt.plot(harmonic_freqs, harmonic_amps, 'ro', markersize=8, label='倍音')
    
    # 倍音番号を注釈
    for freq, amp, num in zip(harmonic_freqs, harmonic_amps, harmonic_numbers):
        plt.annotate(f'{num}', (freq, amp), xytext=(5, 5), 
                    textcoords='offset points', fontsize=10, color='red')
    
    # 基音を強調
    plt.axvline(x=result['fundamental_frequency'], color='orange', linestyle='--', 
               linewidth=2, label=f'基音: {result["fundamental_frequency"]:.1f} Hz')
    
    plt.title(f'{case_name}')
    plt.xlabel('周波数 (Hz)')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 1200)
    
    # 検出結果をコンソールに出力
    print(f"\n{case_name}:")
    print(f"検出された基音: {result['fundamental_frequency']:.1f} Hz")
    print(f"入力基音: {fundamental} Hz")
    print("検出された倍音:")
    for h in result['harmonics']:
        print(f"  {h['harmonic_number']}倍音: {h['frequency']:.1f} Hz, {h['amplitude_db']:.1f} dB")

plt.tight_layout()
plt.show()

## 3. 音色の数値化

In [ ]:
def calculate_timbre_features(harmonics_result):
    """
    倍音構造から音色特徴量を計算
    """
    harmonics = harmonics_result['harmonics']
    
    if len(harmonics) < 2:
        return {}
    
    # 各倍音の正規化振幅を計算
    fundamental_amp = harmonics[0]['amplitude'] if harmonics[0]['harmonic_number'] == 1 else 0
    
    if fundamental_amp == 0:
        return {}
    
    normalized_amps = [h['amplitude'] / fundamental_amp for h in harmonics]
    harmonic_nums = [h['harmonic_number'] for h in harmonics]
    
    # 特徴量計算
    features = {}
    
    # 1. スペクトル重心（明るさの指標）
    weighted_sum = sum(freq * amp for freq, amp in 
                      zip([h['frequency'] for h in harmonics], normalized_amps))
    total_amp = sum(normalized_amps)
    features['spectral_centroid'] = weighted_sum / total_amp
    
    # 2. 偶数倍音 vs 奇数倍音の比率
    even_amp = sum(amp for num, amp in zip(harmonic_nums, normalized_amps) if num % 2 == 0)
    odd_amp = sum(amp for num, amp in zip(harmonic_nums, normalized_amps) if num % 2 == 1)
    features['even_odd_ratio'] = even_amp / (odd_amp + 1e-10)
    
    # 3. 高周波成分の強さ（高い倍音の合計）
    high_harmonics_amp = sum(amp for num, amp in zip(harmonic_nums, normalized_amps) if num >= 5)
    features['high_frequency_content'] = high_harmonics_amp
    
    # 4. 倍音の広がり（標準偏差）
    mean_harmonic = sum(num * amp for num, amp in zip(harmonic_nums, normalized_amps)) / total_amp
    variance = sum(amp * (num - mean_harmonic)**2 for num, amp in zip(harmonic_nums, normalized_amps)) / total_amp
    features['harmonic_spread'] = np.sqrt(variance)
    
    # 5. 基音の相対的強さ
    features['fundamental_strength'] = normalized_amps[0] if harmonic_nums[0] == 1 else 0
    
    return features

# 様々な楽器音色をシミュレート
instrument_profiles = {
    'フルート': [1.0, 0.1, 0.05, 0.02, 0.01, 0.005],
    'クラリネット': [1.0, 0.1, 0.8, 0.1, 0.6, 0.1, 0.4],  # 奇数倍音が強い
    'サクソフォン': [1.0, 0.5, 0.7, 0.4, 0.5, 0.3, 0.4, 0.2],
    'トランペット': [1.0, 0.8, 0.6, 0.7, 0.5, 0.4, 0.3, 0.2],
    'バイオリン': [1.0, 0.7, 0.6, 0.5, 0.4, 0.3, 0.25, 0.2],
    'チェロ': [1.0, 0.6, 0.4, 0.3, 0.2, 0.15, 0.1, 0.08],
    'ピアノ': [1.0, 0.4, 0.3, 0.2, 0.15, 0.1, 0.08, 0.06],
    'オルガン': [1.0, 0.8, 0.6, 0.5, 0.4, 0.35, 0.3, 0.25]
}

# 各楽器の音色特徴を計算
timbre_data = []
fundamental = 220  # A3

for instrument, amplitudes in instrument_profiles.items():
    t, signal = create_harmonic_series(fundamental, len(amplitudes), amplitudes, sample_rate, duration)
    result = detect_harmonics(signal, sample_rate)
    features = calculate_timbre_features(result)
    
    if features:  # 特徴量が計算できた場合
        features['instrument'] = instrument
        timbre_data.append(features)

# 特徴量の可視化
plt.figure(figsize=(16, 12))

# 特徴量のリスト
feature_names = ['spectral_centroid', 'even_odd_ratio', 'high_frequency_content', 
                'harmonic_spread', 'fundamental_strength']
feature_labels = ['スペクトル重心', '偶数/奇数倍音比', '高周波成分', '倍音の広がり', '基音の強さ']

instruments = [data['instrument'] for data in timbre_data]

for i, (feature, label) in enumerate(zip(feature_names, feature_labels)):
    plt.subplot(2, 3, i + 1)
    
    values = [data[feature] for data in timbre_data]
    colors = plt.cm.Set3(np.linspace(0, 1, len(instruments)))
    
    bars = plt.bar(range(len(instruments)), values, color=colors)
    plt.title(f'{label}')
    plt.xticks(range(len(instruments)), instruments, rotation=45, ha='right')
    plt.ylabel('値')
    plt.grid(True, alpha=0.3)
    
    # 値をバーの上に表示
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values) * 0.01,
                f'{value:.2f}', ha='center', va='bottom', fontsize=8)

# 散布図での比較
plt.subplot(2, 3, 6)
x = [data['spectral_centroid'] for data in timbre_data]
y = [data['high_frequency_content'] for data in timbre_data]

plt.scatter(x, y, c=colors, s=100, alpha=0.7)
for i, instrument in enumerate(instruments):
    plt.annotate(instrument, (x[i], y[i]), xytext=(5, 5), 
                textcoords='offset points', fontsize=9)

plt.xlabel('スペクトル重心 (Hz)')
plt.ylabel('高周波成分')
plt.title('楽器の音色特徴空間')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 特徴量の数値表示
print("楽器別音色特徴量:")
print("楽器名\t\tスペクトル重心\t偶数/奇数比\t高周波成分\t倍音の広がり\t基音の強さ")
print("-" * 80)
for data in timbre_data:
    print(f"{data['instrument']:<12}\t{data['spectral_centroid']:.1f}\t\t{data['even_odd_ratio']:.2f}\t\t{data['high_frequency_content']:.2f}\t\t{data['harmonic_spread']:.2f}\t\t{data['fundamental_strength']:.2f}")

## 練習問題

1. 弦楽器と管楽器の倍音構造の違いを比較分析してみましょう
2. 音の高さ（基音）が変わったときの倍音構造の変化を調べてみましょう
3. 実際の楽器音を録音して倍音解析を行い、シミュレーション結果と比較してみましょう